In [10]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(0)

def load_data_from_file(filename = 'data/advertising.csv'):
    data = np.genfromtxt(filename,dtype=None, delimiter=',', skip_header=1)
    features_X = data[:, :3]
    labels_y = data[:, 3]

    intercept = np.ones((features_X.shape[0], 1))
    features_X = np.concatenate((intercept, features_X), axis=1)

    return features_X, labels_y

In [24]:
features_X, sales_y = load_data_from_file()
print(features_X[:5, :])
print(sales_y.shape)

[[  1.  230.1  37.8  69.2]
 [  1.   44.5  39.3  45.1]
 [  1.   17.2  45.9  69.3]
 [  1.  151.5  41.3  58.5]
 [  1.  180.8  10.8  58.4]]
(200,)


In [21]:
def generate_random_value(bound=10):
    return (random.random() - 0.5) * bound

def create_individual(n=4, bound=10):
    individual = [generate_random_value(bound) for _ in range(n)]
    return individual
    
individual = create_individual()
print(individual)

[4.097462559682401, 4.827854760376531, 3.1021723599658957, 4.021659504395827]


In [26]:
def compute_loss(individual):
    theta = np.array(individual)
    y_hat = features_X.dot(theta)
    loss = np.multiply((y_hat - sales_y), (y_hat - sales_y)).mean()
    return loss

def compute_fitness(individual):
    loss =  compute_loss(individual)
    fitness_value = 1 / (loss + 1)

    return fitness_value

individual = [4.09, 4.82, 3.10, 4.02]
loss = compute_fitness(individual)
print(loss)

1.0185991537088997e-06


In [27]:
def crossover(individual1, individual2, crossover_rate=0.9):
    individual1_new = individual1.copy()
    individual2_new = individual2.copy()

    for i in range(len(individual1)):
        if random.random() < crossover_rate:
            individual1_new[i], individual2_new[i] = individual2_new[i], individual1_new[i]

    return individual1_new, individual2_new

individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.4]

individual1, individual2 = crossover(individual1, individual2, 2.0)
print(individual1)
print(individual2)


[3.44, 2.57, -0.79, -2.4]
[4.09, 4.82, 3.1, 4.02]


In [29]:
def mutate(individual, mutation_rate = 0.05):
    individual_mutated = individual.copy()
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual_mutated[i] = generate_random_value()

    return individual_mutated
    
before_individual = [4.09, 4.82, 3.10, 4.02]
after_individual = mutate(before_individual, 2.0)
print(before_individual == after_individual)

False


In [30]:
def selection(sorted_old_population, m=100):
    index1 = random.randint(0, m-1)
    while True:
        index2 = random.randint(0, m-1)
        if (index2 != index1):
            break
    
    individual_s = sorted_old_population[index1]
    if index2 > index1:
        individual_s = sorted_old_population[index2]

    return individual_s

In [ ]:
def create_new_population(old_population, elitism=2, gen=1):
    m = len(old_population)
    sorted_population = sorted(old_population, key=compute_fitness)
    
    if gen%1 == 0:
        print("Best loss:", compute_loss(sorted_population[m-1]), "with chromosome", sorted_population[m-1])
    
    new_population = []
    while len(new_population) < m - elitism:
        # selection
        individual_s1 = selection(sorted_population)
        individual_s2 = selection(sorted_population)

        # crossover
        individual_c1, individual_c2 = crossover(individual_s1, individual_s2)
        
        #mutation
        individual_m1 = mutate(individual_c1)
        individual_m2 = mutate(individual_c2)

        new_population.append(individual_m1)
        new_population.append(individual_m2)
    
    for ind in sorted_population[m-elitism:]:
        new_population.append(ind.copy())
    return new_population, compute_fitness(new_population[m-1])


In [ ]:
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57, -0.79, -2.4]

old_population = [individual1, individual2]
new_population, best_fitness = create_new_population(old_population, elitism=2, gen=1)
print(new_population)
print(best_fitness)